In [1]:
'''

ASSUMPTIONS LIST:

'''

'\n\nASSUMPTIONS LIST:\n\n'

In [2]:
from pypdf import PdfReader
import os
import pyspark as spark
from datetime import datetime

In [26]:
def handle_times(input_text):
    if "P" in input_text and "PM" not in input_text:
        input_text = input_text.replace("P","PM")
    format = '%Y-%m-%d %H:%M %p'
    time = datetime.strptime(input_text, '%I:%M%p')
    list = [time.hour, time.minute]
    return list

In [37]:

files = [f for f in os.listdir('.') if os.path.isfile(f)]
full_text = []
col_heads = []
dates = []
weekdays = ["MONDAY","TUESDAY","WEDNESDAY","THURSDAY","FRIDAY"]
line_is_times = False
times_matrix = []
times_count = 0
for f in files:
    if "finals.pdf" in f:
        reader = PdfReader("finals.pdf")
        number_of_pages = len(reader.pages)
        for page in reader.pages:
            text = page.extract_text(extraction_mode="layout").split('\n')
            text.pop(0)
            for line in text:
                line_is_times = "LOCATION" in line
                if line_is_times:
                    line = line.strip()
                    line = line.split(" ")
                    better_line = []
                    for el in line:
                        if el != '' and el != "LOCATION":
                            better_line.append(el)
                    line = better_line
                    times_line = []
                    for time_range in line:
                        times = time_range.split('-')
                        for time in times:
                            times_line.append(handle_times(time))
                    times_matrix.append(times_line)
                for day in weekdays:
                    not_date_line = True
                    if day in line:
                        not_date_line = False
                        line = line.strip()
                        index = line.index(day)
                        date = line[:index-1]
                        dates.append((day,date))
                        break
                if not_date_line and not line_is_times:
                    line = line.strip()
                    line = line.replace("ALL SECTIONS","ALLSECTIONS").replace(" & ","&").replace("TBA","T BA").replace("ONLINE","ON LINE").replace(" / ","/").replace("ALL SECT","ALLSECT")
                    line = line.split(" ")
                    fixed_line = []
                    deleting = False
                    for word in line:
                        if "(" in word and "ALL" not in word:
                            deleting = True
                        if deleting:
                            if ")" in word:
                                deleting = False
                        else:
                            fixed_line.append(word)
                    line = fixed_line
                    
                    line = [i for i in line if i != '']
                    #print(len(line)/4)
                    if len(line)/5 % 1 != 0:
                        print(line,len(line))#
                    #print(line)
print(dates,len(dates))
print(times_matrix,len(times_matrix))

['MANE', '4610', '01&02', 'JEC', '3232', 'MATH', '1010', 'T', 'BA', 'MANE', '6963', '01', 'T', 'BA'] 14
['MANE', '6660', '01', 'T', 'BA', 'MATH', '2400', 'T', 'BA', 'MATH', '1010', 'T', 'BA'] 13
['ARCH', '2510', '01&03', 'T', 'BA', 'ARCH', '4966', '01', 'T', 'BA', 'BMED', '4250', '01', 'T', 'BA', 'ARCH', '4910', '01,', '02,', '03,', '04,', '05', 'ON', 'LINE'] 24
['ARCH', '4320', '01,', '02,', '03,', '04', 'T', 'BA', 'ARCH', '5100', '01', 'ON', 'LINE', 'BMED', '6420', '01', 'T', 'BA', 'ARCH', '6380', '80', 'T', 'BA'] 23
['CIVL', '2050', '01,', '02,', '03', 'T', 'BA', 'ARCH', '4956', '80', 'T', 'BA', 'CIVL', '4010', '01', 'T', 'BA', 'BCBP', '6345', '01', 'T', 'BA'] 22
['MATH', '2010', 'T', 'BA', 'ARTS', '6900', '01', 'WEST', '113', 'CSCI', '2300', '(ALLSECTIONS)', 'T', 'BA', 'BIOL', '6345', '01', 'T', 'BA'] 19
['MATH', '2400', 'T', 'BA', 'BMED', '2050', '01&02', 'T', 'BA', 'CSCI', '4961', '01', 'T', 'BA', 'CIVL', '4960', '01', 'T', 'BA'] 19
['ENVE', '4240', '01', 'T', 'BA', 'ISYE', '6900